In [1]:
import sys
import torch

'''
pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128
https://developer.nvidia.com/cuda-12-8-0-download-archive
'''

MAC_DIR = '/Users/igwanhyeong/PycharmProjects/ts_forecaster_lib/raw_data/'
WINDOW_DIR = 'C:/Users/USER/PycharmProjects/ts_forecaster_lib/raw_data/'

if sys.platform == 'win32':
    DIR = WINDOW_DIR
    print(torch.cuda.is_available())
    print(torch.cuda.device_count())
    print(torch.version.cuda)
    print(torch.__version__)
    print(torch.cuda.get_device_name(0))
    print(torch.__version__)
else:
    DIR = MAC_DIR
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

save_dir = DIR + 'fit/model_validation'

# if os.path.exists(save_dir):
#     files = glob.glob(os.path.join(save_dir, "*.pt"))
#     print(f"Deleting {len(files)} old checkpoint files...")
#     for f in files:
#         try:
#             os.remove(f)
#         except Exception as e:
#             print(f"Error deleting {f}: {e}")
# else:
#     os.makedirs(save_dir, exist_ok=True)

print("Clean up complete.")


True
1
12.8
2.11.0.dev20260112+cu128
NVIDIA GeForce RTX 5080
2.11.0.dev20260112+cu128
Clean up complete.


In [2]:
import polars as pl
import numpy as np

ETT1 = pl.read_csv(DIR + "csv/ETT/ETTh1.csv")

df = (
    ETT1
    .select(["date", "HUFL"])
    .with_columns(pl.lit("A").alias("unique_id"))
    # 원본 date 문자열을 그대로 Datetime으로 파싱
    .with_columns(
        pl.col("date").str.strptime(pl.Datetime, format="%Y-%m-%d %H:%M:%S", strict=False).alias("date")
    )
    .sort(["unique_id", "date"])
)

# time index
df = df.with_columns(
    pl.arange(0, pl.len()).over("unique_id").alias("t_idx")
)

# (1) known-future 스케줄: promo (예: 특정 시간대에만 1)
# 하루 24시간 중 8~10시, 18~20시에 프로모션이라고 가정
df = df.with_columns([
    (pl.col("t_idx") % 24).alias("hour"),
])

df = df.with_columns([
    (
        ((pl.col("hour") >= 8) & (pl.col("hour") <= 10)) |
        ((pl.col("hour") >= 18) & (pl.col("hour") <= 20))
    ).cast(pl.Int8).alias("promo_flag")
])

# (2) calendar exo: 24h sin/cos
df = df.with_columns([
    ( (2*np.pi*pl.col("t_idx")/24.0).sin().cast(pl.Float32) ).alias("exo_fut_sin24"),
    ( (2*np.pi*pl.col("t_idx")/24.0).cos().cast(pl.Float32) ).alias("exo_fut_cos24"),
])

# (3) (중요) 타깃에 promo 효과 "주입" -> exo가 없으면 예측이 어려워지고, 있으면 쉬워짐
# HUFL_y = HUFL + alpha*promo_flag + beta*sin24  (alpha는 체감되게 크게)
alpha = 2.0
beta  = 0.5
df = df.with_columns([
    (
        pl.col("HUFL").cast(pl.Float32)
        + pl.col("promo_flag").cast(pl.Float32) * pl.lit(alpha)
        + pl.col("exo_fut_sin24").cast(pl.Float32) * pl.lit(beta)
    ).alias("y")
])

# =========================
# past_exo 후보 생성
# =========================
# 기준: y를 만들었으면 y 기반으로 만드는 게 가장 직관적.
# (HUFL 원본 기반으로도 가능하나, 지금은 y에 promo/seasonality가 주입되어 있으니 y 기준 추천)

df = df.with_columns([
    # (A) lag / diff
    pl.col("y").shift(1).over("unique_id").alias("pe_lag1_y"),
    pl.col("y").shift(24).over("unique_id").alias("pe_lag24_y"),  # 하루 전(24시간 전)
    (pl.col("y") - pl.col("y").shift(1).over("unique_id")).alias("pe_diff1_y"),
    (pl.col("y") - pl.col("y").shift(24).over("unique_id")).alias("pe_diff24_y"),

    # (B) rolling mean / std (짧은/중간 윈도우)
    pl.col("y").rolling_mean(window_size=6).over("unique_id").alias("pe_rm6_y"),
    pl.col("y").rolling_mean(window_size=24).over("unique_id").alias("pe_rm24_y"),
    pl.col("y").rolling_std(window_size=24).over("unique_id").alias("pe_rs24_y"),

    # (C) z-score (24시간 기준)
    (
        (pl.col("y") - pl.col("y").rolling_mean(24).over("unique_id"))
        / (pl.col("y").rolling_std(24).over("unique_id") + 1e-6)
    ).alias("pe_z24_y"),

    # (D) EMA (지수이동평균) - Polars ewm_mean 사용
    pl.col("y").ewm_mean(alpha=0.2).over("unique_id").alias("pe_ema_a02_y"),

    # (E) promo의 과거 상태 (이벤트의 lag)
    pl.col("promo_flag").shift(1).over("unique_id").cast(pl.Float32).alias("pe_lag1_promo"),
    pl.col("promo_flag").rolling_mean(24).over("unique_id").cast(pl.Float32).alias("pe_rm24_promo"),
])

# rolling/shift로 인해 처음 구간에 null이 생깁니다.
# TrainingDataset은 null을 그대로 numpy로 가져오면 nan이 될 수 있으니, 보통 0으로 채우는 편이 안전합니다.
past_cols = [
    "pe_lag1_y", "pe_lag24_y", "pe_diff1_y", "pe_diff24_y",
    "pe_rm6_y", "pe_rm24_y", "pe_rs24_y", "pe_z24_y",
    "pe_ema_a02_y", "pe_lag1_promo", "pe_rm24_promo",
]

df = df.with_columns([pl.col(c).fill_null(0.0).cast(pl.Float32) for c in past_cols])

df.select(["date","promo_flag", "y", 'HUFL'] + past_cols).head(5)


FileNotFoundError: 지정된 파일을 찾을 수 없습니다. (os error 2): C:/Users/USER/PycharmProjects/ts_forecaster_lib/raw_data/csv/ETTh1.csv

In [ ]:
from modeling_module.utils.metrics import mae, smape

from modeling_module.utils.metrics import rmse


def load_model_ckpt(model, ckpt_path: str, device: str):
    state = torch.load(ckpt_path, map_location="cpu")
    model.load_state_dict(state["model_state"], strict=True)
    model.to(device)
    model.eval()
    return model

@torch.no_grad()
def eval_on_loader(model, loader, device: str):
    model.eval()
    ys, yhats = [], []

    print('batch_size', loader.batch_size)
    for batch in loader:
        x = batch[0].to(device)              # [B, 52, 1]
        y = batch[1].to(device)              # [B, 27]
        future_exo = batch[3].to(device)     # [B, 27, 4]
        past_exo_cont = batch[4].to(device)  # [B, 52, 11]
        past_exo_cat = batch[5].to(device)   # [B, 52, 0]

        # PatchTSTPointModel.forward 시그니처에 맞춰 전달
        yhat = model(
            x,
            future_exo=future_exo,
            past_exo_cont=past_exo_cont,
            past_exo_cat=past_exo_cat,
        )  # [B, 27]

        ys.append(y.detach().cpu().numpy())
        yhats.append(yhat.detach().cpu().numpy())

    y_all = np.concatenate(ys, axis=0)
    yhat_all = np.concatenate(yhats, axis=0)
    return y_all, yhat_all


def _extract_pred_from_output(out, prefer_q=0.5):
    """
    Quantile 모델 출력(out)이 dict/tuple/tensor 등일 수 있으므로
    예측 텐서를 안전하게 꺼낸다.

    반환:
      - yhat_point: [B,H] (예: q=0.5 또는 첫 번째 출력)
      - extra: 원본 out (필요 시 분석용)
    """
    # 1) Tensor면 그대로
    if torch.is_tensor(out):
        return out, out

    # 2) tuple/list면 첫 원소를 예측으로 가정
    if isinstance(out, (tuple, list)):
        # 가장 흔한 케이스: (pred, aux) 또는 (q_pred, ...)
        first = out[0]
        if torch.is_tensor(first):
            return first, out
        # 더 복잡하면 아래 dict 로직으로 넘기기 위해 out를 dict처럼 처리 불가 -> 에러
        raise TypeError(f"Unsupported tuple/list output types: {[type(x) for x in out]}")

    # 3) dict면 키 후보들에서 찾기
    if isinstance(out, dict):
        # 흔한 키 후보들
        key_candidates = [
            "yhat", "pred", "prediction", "y_pred", "output",
            "q_pred", "quantiles", "yq", "y_hat"
        ]
        for k in key_candidates:
            if k in out and torch.is_tensor(out[k]):
                t = out[k]
                # [B,H] 또는 [B,H,Q] 또는 [B,Q,H] 등 가능
                return _select_point_from_quantile_tensor(t, prefer_q=prefer_q), out

        # dict 안에 텐서가 하나뿐이면 그걸 쓰기
        tensor_items = [(k, v) for k, v in out.items() if torch.is_tensor(v)]
        if len(tensor_items) == 1:
            t = tensor_items[0][1]
            return _select_point_from_quantile_tensor(t, prefer_q=prefer_q), out

        raise KeyError(f"Cannot find tensor prediction in dict output. keys={list(out.keys())}")

    raise TypeError(f"Unsupported model output type: {type(out)}")


def _select_point_from_quantile_tensor(t: torch.Tensor, prefer_q=0.5):
    """
    t가
      - [B,H]이면 그대로
      - [B,H,Q]이면 Q축에서 median(0.5)에 해당하는 index 선택
      - [B,Q,H]이면 Q축에서 선택 후 [B,H]로
    """
    if t.ndim == 2:
        return t

    if t.ndim == 3:
        B, d1, d2 = t.shape

        # [B,H,Q] 케이스 가정: d1이 horizon, d2가 quantile
        # [B,Q,H] 케이스 가정: d1이 quantile, d2가 horizon
        # heuristic: horizon은 보통 27 같은 값, quantile은 보통 3/5/9 등 작은 값
        if d1 > d2:
            # [B,H,Q]
            q_dim = 2
            q_len = d2
            h_dim = 1
        else:
            # [B,Q,H]
            q_dim = 1
            q_len = d1
            h_dim = 2

        # prefer_q=0.5에 해당하는 index를 선택 (가능하면 중앙)
        # q 값 리스트를 out에 포함하지 않는 경우가 많으니 중앙 index를 사용
        q_idx = int(round((q_len - 1) * prefer_q))  # median index

        if q_dim == 2:
            # [B,H,Q] -> [B,H]
            return t[:, :, q_idx]
        else:
            # [B,Q,H] -> [B,H]
            return t[:, q_idx, :]

    raise ValueError(f"Unexpected prediction tensor shape: {t.shape}")


@torch.no_grad()
def eval_on_loader_quantile(model, loader, device: str, prefer_q: float = 0.5):
    """
    Quantile 모델 평가:
      - y_true: [N,H]
      - yhat_point: [N,H] (기본: 중앙 quantile로 point화)
      - outs: 필요 시 분석용 raw output list(옵션으로 쓸 수 있음)
    """
    model.eval()
    ys, yhats = [], []

    for batch in loader:
        x = batch[0].to(device)              # [B, 52, 1]
        y = batch[1].to(device)              # [B, 27]
        future_exo = batch[3].to(device)     # [B, 27, 4]
        past_exo_cont = batch[4].to(device)  # [B, 52, 11]
        past_exo_cat = batch[5].to(device)   # [B, 52, 0]

        out = model(
            x,
            future_exo=future_exo,
            past_exo_cont=past_exo_cont,
            past_exo_cat=past_exo_cat,
        )

        yhat_point, _ = _extract_pred_from_output(out, prefer_q=prefer_q)

        ys.append(y.detach().cpu().numpy())
        yhats.append(yhat_point.detach().cpu().numpy())

    y_all = np.concatenate(ys, axis=0)
    yhat_all = np.concatenate(yhats, axis=0)
    return y_all, yhat_all

In [ ]:
from modeling_module.data_loader import MultiPartExoDataModule
from modeling_module.utils.exogenous_utils import compose_exo_calendar_cb
import random
import numpy as np

lookback = 27
horizon = 8

def set_seed(seed = 11):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.backends.mps.is_available():
        torch.mps.manual_seed(seed)

set_seed(11)
save_root_A = DIR + 'fit/model_validation/A_no_pretrain'
save_root_B = DIR + 'fit/model_validation/B_with_pretrain'


future_exo_cb = compose_exo_calendar_cb(date_type = 'H', sincos = True)

data_module = MultiPartExoDataModule(
    df,
    id_col = 'unique_id',
    date_col = 'date',
    y_col = 'y',
    lookback = lookback,
    horizon = horizon,
    batch_size = 128,
    past_exo_cont_cols = past_cols,
    future_exo_cb = future_exo_cb,
    freq = 'hourly',
    shuffle = True,
    split_mode = 'multi',
)

train_loader = data_module.get_train_loader()
val_loader = data_module.get_val_loader()

In [ ]:
from modeling_module.training.model_trainers.total_train import run_total_train_hourly, summarize_metrics

results_A = run_total_train_hourly(
    train_loader,
    val_loader,
    device=device,
    lookback=lookback,
    horizon=horizon,
    save_dir=save_root_A,
    models_to_run=["patchtst"],
    use_ssl_pretrain=False,         # ★ A: off
)

In [ ]:
from modeling_module.training.model_trainers.total_train import run_total_train_hourly, summarize_metrics


results_B = run_total_train_hourly(
    train_loader,
    val_loader,
    device=device,
    lookback=lookback,
    horizon=horizon,
    save_dir=save_root_B,
    models_to_run=["patchtst"],
    use_ssl_pretrain=True,          # ★ B: on
    ssl_pretrain_epochs=10,
    ssl_mask_ratio=0.3,
    ssl_loss_type="mse",
    ssl_freeze_encoder_before_ft=False,  # freeze->unfreeze는 다음 단계에서
)


In [ ]:
from modeling_module.utils.checkpoint import load_model_dict
from modeling_module.models import build_patchTST_base, build_patchTST_quantile

builders = {
    "patchtst_quantile": build_patchTST_quantile,
}

modelA = load_model_dict(f"{save_root_A}", builders, device = device)['patchtst_quantile']
modelB = load_model_dict(f"{save_root_B}", builders, device = device)['patchtst_quantile']

# yA, yhatA = eval_on_loader(modelA, val_loader, device)
# yB, yhatB = eval_on_loader(modelB, val_loader, device)

yA, yhatA = eval_on_loader_quantile(modelA, val_loader, device, prefer_q=0.5)
yB, yhatB = eval_on_loader_quantile(modelB, val_loader, device, prefer_q=0.5)

metricA = {
    "MAE": float(mae(yA.reshape(-1), yhatA.reshape(-1))),
    "RMSE": float(rmse(yA.reshape(-1), yhatA.reshape(-1))),
    "SMAPE": float(smape(yA.reshape(-1), yhatA.reshape(-1))),
}
metricB = {
    "MAE": float(mae(yB.reshape(-1), yhatB.reshape(-1))),
    "RMSE": float(rmse(yB.reshape(-1), yhatB.reshape(-1))),
    "SMAPE": float(smape(yB.reshape(-1), yhatB.reshape(-1))),
}
metricA, metricB


In [ ]:
import matplotlib.pyplot as plt

def plot_one_sample(y_true, yhatA, yhatB, idx=0):
    yt = y_true[idx].reshape(-1)
    ya = yhatA[idx].reshape(-1)
    yb = yhatB[idx].reshape(-1)

    plt.figure()
    plt.plot(yt, label="true")
    plt.plot(ya, label="A_no_pretrain")
    plt.plot(yb, label="B_with_pretrain")
    plt.legend()
    plt.title(f"A/B forecast comparison (sample={idx})")
    plt.show()

plot_one_sample(yA, yhatA, yhatB, idx=128)

In [ ]:
import matplotlib.pyplot as plt

def plot_all_samples_vertical(y_true, yhatA, yhatB, max_n=128):
    n = min(max_n, y_true.shape[0])

    fig, axes = plt.subplots(
        nrows=n, ncols=1,
        figsize=(12, 2.2 * n),   # n이 커질수록 세로 길이 증가
        sharex=True,
        sharey=False
    )

    # n=1일 때 axes가 단일 객체가 되므로 list로 통일
    if n == 1:
        axes = [axes]

    for i in range(n):
        yt = y_true[i].reshape(-1)
        ya = yhatA[i].reshape(-1)
        yb = yhatB[i].reshape(-1)

        ax = axes[i]
        ax.plot(yt, label="true")
        ax.plot(ya, label="A_no_pretrain")
        ax.plot(yb, label="B_with_pretrain")
        ax.set_title(f"sample={i}", fontsize=9)

        # 너무 복잡해지니 legend는 첫 번째 축에만
        if i == 0:
            ax.legend(loc="upper right", fontsize=8)

    fig.suptitle("A/B forecast comparison (all samples)", y=1.002, fontsize=12)
    plt.tight_layout()
    plt.show()

plot_all_samples_vertical(yA, yhatA, yhatB, max_n=128)
